# Extracting Features From the Original Dataset
This notebook is used to load my top songs playlists from 2016 to 2022 and store the into a `.cvs` file.


In [ ]:
# Import libraries
import re

import pandas as pd
import spotipy
import spotipy.util as util
from tqdm import tqdm

We use the credentials saved into config.py to input as arguments to `util.prompt_for_user_token`

In [ ]:
# File containing credentials
import config
# Get user authorization to read a user’s saved tracks:
token = util.prompt_for_user_token(username='11182181486',
                                   scope='user-library-read',
                                   client_id=config.cid,
                                   client_secret=config.secret,
                                   redirect_uri=config.redirect_url)
sp = spotipy.Spotify(auth=token)

In [ ]:
# Function to retrieve data for each song from Spotify Web API
def playlist_df(uri):
    ## Extracting Tracks From a Playlist
    track_uris = sp.playlist_items(uri)["items"]
    ## Create lists to save songs details
    artist_name = []
    artist_pop = []
    artist_genres = []
    track_name = []
    track_id = []
    track_uri = []
    popularity = []
    # for track in sp.playlist_tracks(uri)["items"]:
    for track in track_uris:
        # Track URI
        track_uri.append(track["track"]["uri"])
        # Track ID
        track_id.append(track["track"]['id'])
        # Track name
        track_name.append(track["track"]["name"])

        # Main Artist
        artist_uri = track["track"]["artists"][0]["uri"]
        artist_info = sp.artist(artist_uri)
        # Name, popularity, genre
        artist_name.append(track["track"]["artists"][0]["name"])
        artist_pop.append(artist_info["popularity"])
        artist_genres.append(artist_info["genres"])

        # Popularity of the track
        popularity.append(track["track"]["popularity"])

    # Create dataframe
    tracks_df = pd.DataFrame(
        {'artist_name': artist_name, 'artist_pop': artist_pop, 'artist_genres': artist_genres,
         'track_name': track_name, 'track_id': track_id, 'track_uri': track_uri, 'popularity': popularity})

    # Edit the track-uris to a more usable format
    tracks_df["track_uri"] = tracks_df["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
    # Get songs features
    feature_list = []
    for track in tracks_df["track_uri"]:
        feature_list.append(sp.audio_features(track)[0])
    # Create DataFrame
    feature_df = pd.DataFrame(feature_list)
    # Merge dataframes
    df = pd.merge(tracks_df, feature_df, left_on="track_uri", right_on="id")
    return df

In [ ]:
# Load Data into DataFrame for Exploratory Data Analysis
# Create empty dataframe that will contain all the details
# Import file containing links
import playlists
data = pd.DataFrame()
print("Extract songs features from each playlist:")
for key, value in tqdm(playlists.links.items()):
    # Get playlists URI
    playlist_URI = value.split("/")[-1].split("?")[0]
    # Get dataframe with the features of each song
    df = playlist_df(value)
    # Add column representing year
    df['year'] = int(key)
    # Add dataframe to data
    data = pd.concat([data, df])
print("Done")
print(data.shape)

In [ ]:
data.head()

In [ ]:
## Save dataframe as .cvs to data folder
data.to_csv('./data/my_top_songs.csv')

The final cvs file can be found in the data [repository](https://github.com/zpapagianni/Spotify_project/blob/15ecd506ef459e4b20537a0d861286df083929e2/data). Next we will move to Part 2:Exploratory data analysis.